In [1]:
import json
import pandas as pd
data = None
with open('data_final_final.json', encoding='utf-8') as file:
    data = json.load(file)

In [2]:
from multiprocessing.dummy import Pool
import re
import time
tempTime = None
__keyWordWhiteList__ = [
    '新冠', '病毒', '口罩', '武汉', '感染',
    '疫情', '中国', '方舱', '钟南山', '火神山',
    '雷神山', '隔离', '新增确诊', '封城', '解封',
    '疫苗'
]
__userBlackList__ = [
    '搞笑', '金融', '保险', '香雪儿菲',
    '报道', '爆料', '发生', '微生活', '新鲜事', '律师', '独乐乐不如众乐涛', '球球追热剧',
    '共青团', '每日忘忧', '冷饭热茶', '华尔街', '天道求自在', '主播', '新东方', '在线',
    '创业', '顾问', '泫雅', '潮牌'
]
__topicBlackList__ = [
    '1009', '若松问凝', '昊嘉笑柳', 'cgd', '💟L澳门宏利保险-梁雪梅Jennifer的微博视频',
    '%', 'arabica', '/', '康辉旅游', 'tfboys', '蔡徐坤', '肖战', '王一博', '周冬雨',
    'NBA', '碟中谍', '蔷薇花开0321', '火箭少女', '优学资讯站', '王牌解说',
    '[0-9]{2,}$', '大龄女青年', '张艺兴', '晨涛若山', '昌盛若芹', '恋爱日记',
    '情感语录', '陇南旅行', '真珠美学', '巧兰浦泽', '家居装修', '独立女性',
    '情感挽回', '医学美眼', '双眼皮', '恋爱物语', '感动瞬间', '楼市',
    'OOTD', '口罩头像', '黄子弘凡', '每日运势', '武汉大学生租房', '武汉租房', '女生头像',
    '中国宁波网', '口罩妆万能公式', '金韩彬', '旅游攻略', '旅行攻略', '阿云嘎', '房地产',
    '爆料', '杨颖', '纪录片人生第一次', '微博视频', '来源：', '有两副面孔的狗子', '武功山',
    '今日问题回答', '何洛洛', '昊磊千凡', '瓜子不是子i5', '陈伟霆留胡子', '看盘', '朱一龙',
    '每日一呼', '抗敏', '热力兄弟', '设计美学', '欧冠', '欧洲杯', '国际疫情动态', '昊天念薇',
    '比亚迪', '中村伦也', '冷笑话', '微博抽奖', '共青团', '名单更新', '回到苏州的第29天',
    '男子在赴美航班上死亡', '头条文章', '滚动更新', '当代男艺人居家隔离状态', '环球网', '环球时报',
    '口罩猫', '身边事', '发布会', '疫情相关', '疫情消息', '邢台学院初等教育学院抗击疫情以读攻毒',
    '通知》', '《关于', '【.*?】', '青年主讲人', '权威回应', '一起走向春天', '李璞', '车模',
    '每日一图', '长安福特', '拥抱今天', '汪汪姐美妆', '一图读懂', '肖木兵速写', '云聘会', '婚礼',
    '暖色暖食', '攻略', '奥运会', '篮网', '李现', '宋亚轩', '走秀', '戴口罩自拍大赛',
    '记录世界.*?记录你', '詹雯婷', '《科学》', '《自然》', '卫生巾', '今日贴纸打卡', '恭喜马老师',
    '马云蝉联中国首富', '美食', '创业', '集美', '.*?记录生活', '徐娇', '指南', '痘痘', '中国科普月',
    '带着边框一起拍', '近视', '预防近视', '全县人民', '志愿者协会', '投资理财', '霍乱',
    '余甜', '关于转发', '辩论', '猫咪表情包', '吸猫', '子宫腺肌病', '子宫肌瘤', '海扶医院',
    '刘雨昕', '我省', '我市', '我区', '我县', '阿乐制茶', '数据更新', '出入平安', '颜值变化',
    '健身', '蜜桃臀', '华晨宇', '买口罩的主力军', '泫雅', '歌声助力', '宝宝的少女心',
    '疫情结束后.*?生们?的.*?', '尤长靖', '管栎', '千与千寻背景图', 'LOFT公寓', '二月初二龙抬头',
    '小满壁纸', '明星势力榜', '王海轮', '吃货无限战疫', '现金红包', '金泰亨', '喵星人',
    '日电', '牛骏峰', '情侣口罩', '女孩子追星的后果', '口罩自拍大赏', '上证指数', '深证成指',
    '纳斯达克', '张杰', '木香那年香', '粉丝公益季', '林彦俊', '拼车人找车', '跟新浪看热点', 
    '薛之谦', '房产', '足球', '我的推荐', '英超', '彭于晏', '娱乐明星', '咖喱酱', '职业学院',
    '李健', '阿森纳', '转发微博', '分享了?话题', '男性健康', '精子畸形', '科比', 'Gigi',
    '.*?卫视', '恋爱', '白敬亭', '李权哲', '剩女', '星座', '情感故事', '情商心理学',
    '魔兽', '高消费令', '滴滴网约车', '滴滴司机可拒载不戴口罩乘客', '奶油酱',
    '漫威', '搞笑幽默', '运珧向松', '杨芸晴', '深圳城中村硬核防疫', '杨舒平', '热点暴击',
    '贺同学的路上见闻', '化妆', '公益海报', '记忆地平线', '当可爱的人遇到洒水车', '12315',
    '南墙不是墙', '雪碧炸鸡', '治愈不开心', '疫情.*?分手', '炒股', '口罩妆', '陈月末', '博主日记',
    '西下几时回', '#.*?动漫.*?#', '摄影', '尤文', '感动', '梁钰', '葡萄酒', '留学百问',
    '金宇彬', '郑秀晶', '釜山行', '每天一个卷腹', '冒个泡', '瘦身', '好省', '青春有你',
    '蛮荒的童话', '图库影像馆', '日赚', '向往的生活', '徐静波', '即日起', '疫情.*?动态', 
    '涠洲岛', '打卡', '华侨大学', '商务楼宇', '无心法师3', '菠萝and梨呀',
    '风衣', 'R1SE', '金泰熙', '新浪网》', '舞蹈益起来', '认真的告白有多甜', '爆裂风车',
    '带着微博去旅行', '曼谷', '周峻纬', '天津红桥区彰武楼小区拾荒老人被推搡殴打', '#.*?微信解封.*?#',
    '全民旅游', '微博收藏', '微博文玩', '天水记录', '情人节', '易烊千玺', '文旅资讯', '王俊凯',
    '吴磊'
]
__keyWordBlackList__ = [
    '猪瘟', '阮苏', '沈沐宸', 'tfboys', 'TFBOYS',
    'TFboys', 'UID593', '今日壁纸库', '^重磅', '爆料',
    '融媒体中心', '分享单曲', '@网易云音乐', '@中国社会福利基金会', '#股票#',
    '#旅行#', '#最新#', '平板支撑', '帕梅拉', '#漫画#', '#成年#'
] + __topicBlackList__
length = 0
def cleanPost(post: dict):
    global __keyWordWhiteList__, __keyWordBlackList__, __userBlackList__, __topicBlackList__
    if post['content'][-5:] == '收起全文d':  # 删除“收起全文”标记
        post['content'] = post['content'][:-5]
    post['content'] = re.sub('O网页链接', '', post['content'])
    if str(type(post['uVIP'])) == "<class 'list'>":  # 将空字符串列表转化为空字符串
        post['uVIP'] = ''
    if str(type(post['collectTime'])) == "<class 'float'>":  # 时间戳转化为时间元组
        post['collectTime'] = time.localtime(post['collectTime'])
    elif str(type(post['collectTime'])) == "<class 'list'>":
        post['collectTime'] = tuple(post['collectTime'])
    if str(type(post['time'])) == "<class 'str'>":  # 发布时间的处理
        if re.findall('今天', post['time']):
            post['time'] = time.strptime(str(post['collectTime'][0]) + '年' + str(post['collectTime'][1]) + '月' + str(
                post['collectTime'][2]) + '日 ' + re.sub('今天', '', post['time']), '%Y年%m月%d日  %H:%M')
        elif re.findall('-.*?-', post['time']):
            post['time'] = time.strptime(
                re.sub('^(.*?); ', '', post['time']), '%Y-%m-%d %H:%M')
        elif re.findall('年.*月.*日', post['time']):
            post['time'] = time.strptime(
                re.sub('^(.*?); ', '', post['time']), '%Y年%m月%d日 %H:%M')
        elif re.findall('月.*日', post['time']):
            post['time'] = time.strptime(
                '2020年' + re.sub('^(.*?); ', '', post['time']), '%Y年%m月%d日 %H:%M')
        elif re.findall('分钟', post['time']):
            post['time'] = time.localtime(time.mktime(tuple(post['collectTime'])) - 60 * int(re.findall('^(.*?)分钟', post['time'])[0]))
        elif re.findall('秒', post['time']):
            post['time'] = time.localtime(time.mktime(tuple(post['collectTime'])) - int(re.findall('^(.*?)秒', post['time'])[0]))
        else:
            return None
    if (str(type(post['time'])) == "<class 'time.struct_time'>") and (time.mktime(post['time']) < 1577635200):
        # 移除早于疫情爆发时期的微博
        return None
    if post['uVIP'] == 'b' or post['uVIP'] == 'g':
        return None
    post['content'] = re.sub('\u200b', '', post['content'])
    post['content'] = re.sub('(:&nbsp;){1,}', '\n', post['content'])
    flagWhite = True
    flagBlack = True
    # 移除不含白名单关键词的微博
    # for keyWord in __keyWordWhiteList__:
    #     if keyWord in post['content']:
    #         flagWhite = True
    # 移除含有黑名单关键词的微博
    length = len(re.sub('#.*?#', '', post['content']))
    if length <= 5 or length > 500:
        flagBlack = False
    for keyWord in __keyWordBlackList__:
        if re.findall(keyWord, post['content']):
            flagBlack = False
        if not flagBlack:
            break
    for topic in __topicBlackList__:
        for keyWord in post['keyWords']:
            if re.findall(topic, keyWord):
                flagBlack = False
            if not flagBlack:
                break
        if not flagBlack:
            break
    for user in __userBlackList__:
        if re.findall(user, post['uName']):
            flagBlack = False
        if not flagBlack:
            break
    if flagWhite and flagBlack:
        return post
    else:
        return None

def cleanList(data: list) -> list:
    pool = Pool(32)
    return list(filter(None, pool.map(cleanPost, data)))

In [3]:
cleaned = cleanList(data)
len(cleaned)

477664

In [4]:
import random
sample = pd.DataFrame(random.choices(cleaned, k=60))[['link', 'content']]
sample

,link,content
0,https://weibo.com/1807808822/IzanyFer2,为啥你们口罩带得都耳朵痛，可我从来没有耳朵痛呀，我晓得了，可能是你们脸大。#戴口罩专用发型#
1,https://weibo.com/3312371651/IuPkz4Psi,#抗疫期间这些行为都是犯罪#所有的事情不应该一概而论，有些地方的防疫执法人员确实存在过度执法...
2,https://weibo.com/2657951717/IwGwH3D2L,#英国新增12名新冠肺炎感染者#希望课程结束的时候能顺利回国🙏英国人真的太爱聚会了，公共场所...
3,https://weibo.com/5585448745/Irychoz1k,#山西新增7例确诊病例#作为山西人吃过最野的可能就是带鱼和虾了那些野味看了都害怕谁敢吃可总有...
4,https://weibo.com/5024399661/Iv4IAAzvU,#疫情过后最想去哪里#我哪也不想去！我还能再宅一个月！
5,https://weibo.com/5497282113/IyZJ2pPcG,囤货的不少带口罩的没几个另外！今天天气太好啦！#海外抗疫日记##澳大利亚疫情#2澳大利亚·墨尔本
6,https://weibo.com/3991556227/Izw7H2Dk0,#疫情快走#我想出去玩O绿洲
7,https://weibo.com/1599229910/Iwbrc6AXS,#又一名伊朗高官确诊新冠肺炎#伊朗副总统也被感染了！我怀疑他们上升到自然选择程度了！
8,https://weibo.com/6815750514/IvnJPBvnz,#韩国疫情#一定要重视加油啊🙏🙏🙏
9,https://weibo.com/6506028372/Ixk8k4Fsg,#韩国禁止在疫情期间出口医疗物资#可以理解，起码要把自己国家保住才行


In [5]:
with open(r'C:\Users\huang\OneDrive - mails.tsinghua.edu.cn\文档\上课\大二下学期\数据挖掘：方法与应用\HW01_2.json',
    'w', encoding='utf-8') as file:
    json.dump(cleaned, file, ensure_ascii=False)